In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
""" First, reload the data """
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']    
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']    
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save # hint to help gc free up memory
    print('Training info:', train_dataset.shape, train_labels.shape)
    print('Valid info:', valid_dataset.shape, valid_labels.shape)
    print('Test info:', test_dataset.shape, test_labels.shape)

Training info: (20000, 28, 28) (20000,)
Valid info: (10000, 28, 28) (10000,)
Test info: (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size,image_size, num_channels)).astype(np.float32)
    # map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0, ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  # one-hot
    return dataset,labels

train_dataset,train_labels = reformat(train_dataset,train_labels)
valid_dataset,valid_labels = reformat(valid_dataset,valid_labels)
test_dataset,test_labels = reformat(test_dataset,test_labels)

print('Training info:', train_dataset.shape, train_labels.shape)
print('Valid info:', valid_dataset.shape, valid_labels.shape)
print('Test info:', test_dataset.shape, test_labels.shape)

Training info: (20000, 28, 28, 1) (20000, 10)
Valid info: (10000, 28, 28, 1) (10000, 10)
Test info: (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    accuracy = (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1)) / predictions.shape[0])
    return accuracy

In [66]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.zeros([depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.zeros([num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.zeros([num_labels]))
    
    def model(data, PREDICT=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv+layer1_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden,keep_prob=0.5)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv+layer2_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden,keep_prob=0.7)

        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden, 0.8)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    def model_pool(data, PREDICT=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME')
        pool = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1],padding='SAME')
        hidden = tf.nn.relu(pool+layer1_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden,keep_prob=0.5)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1,1,1,1], padding='SAME')
        pool = tf.nn.max_pool(conv,[1,2,2,1],[1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(pool+layer2_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden,keep_prob=0.7)

        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        if not PREDICT:
            hidden = tf.nn.dropout(hidden, 0.8)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    train_model = model_pool
    logits = train_model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))

    cur_step = tf.Variable(0)
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate,global_step=cur_step,decay_steps=10000,decay_rate=0.96,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=cur_step)
#     optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(train_model(tf_valid_dataset, PREDICT=True))
    test_prediction = tf.nn.softmax(train_model(tf_test_dataset, PREDICT=True))

In [68]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:offset+batch_size, :, :, :]
        batch_labels = train_labels[offset:offset+batch_size,:]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _,l,predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if step%100==0:
            print('Minibatch Loss at step %d: %f' % (step, l))
            print('Minibatch Training accuracy: %.f%%' % accuracy(predictions, batch_labels))
            print('Minibatch Validation accuracy:%.1f%%' % accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy:%1.f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch Loss at step 0: 2.561693
Minibatch Training accuracy: 12%
Minibatch Validation accuracy:9.3%
Minibatch Loss at step 100: 1.456724
Minibatch Training accuracy: 44%
Minibatch Validation accuracy:63.9%
Minibatch Loss at step 200: 0.797898
Minibatch Training accuracy: 75%
Minibatch Validation accuracy:73.6%
Minibatch Loss at step 300: 0.286605
Minibatch Training accuracy: 100%
Minibatch Validation accuracy:76.5%
Minibatch Loss at step 400: 1.524935
Minibatch Training accuracy: 56%
Minibatch Validation accuracy:78.1%
Minibatch Loss at step 500: 1.048518
Minibatch Training accuracy: 50%
Minibatch Validation accuracy:80.1%
Minibatch Loss at step 600: 0.724149
Minibatch Training accuracy: 75%
Minibatch Validation accuracy:80.6%
Minibatch Loss at step 700: 0.452690
Minibatch Training accuracy: 88%
Minibatch Validation accuracy:80.9%
Minibatch Loss at step 800: 0.830807
Minibatch Training accuracy: 62%
Minibatch Validation accuracy:81.3%
Minibatch Loss at step 900: 0.420093